In [ ]:
%%HTML
<link rel="stylesheet" type="text/css" href="css/rise.css">

In [ ]:
#jupyter nbconvert "presentation.ipynb" --to slides --reveal-prefix "https://cdnjs.cloudflare.com/ajax/libs/reveal.js/3.1.0"

<center><font size="12" color="#efeff0"><b>PyCaretでkaggleにチャレンジ中</b></font></center>
<br>
<center><font size="6" color="#efeff0"><b>Pythonデータ分析勉強会＃19　2020/5/16(土)</b></font></center>

# お前、誰よ？

<table>
    <tr>
        <td><img src="image/fish.gif" alt="Drawing" style="width: 200px;"/></td>
        <td><font size="6">名前　 　：高橋かずひと<br>
            Twitter　：@KzhtTkhs<br>
            お仕事 　：組込、画像処理、IoT、Webアプリなど</font><br>
            <font size="4">その他、個人の活動で、インディーゲーム制作のお手伝をしています。</font>
        </td>
    </tr>
</table>

# 最近はディープラーニング(主に物体検出)やＡＲでよく遊んでいます。

<center><video width="70%" src="image/2019production.mp4" controls/></center>

# 業務命令

上長「高橋くん機械学習(※何を指しているか謎)出来たよね？」<br>
高橋「機械学習(画像系)出来なくもなくもないです」

上長「全社取り組みでAI注力することになったので、
<br>　　　kaggleでメダル取ってください」

上長「あと後輩くん(知識無し)もサクッと育ててほしい」

高橋「そんなバカな🤮」　※ノンノンフィクション

# PyCaretって？

「PyCaret is an open source, low-code  machine learning library in Python that allows you to go from preparing your data to deploying your model within seconds in your choice of notebook environment.」
<br><br>
DeepL翻訳：PyCaretはオープンソースのPythonによるローコード機械学習ライブラリで、データの準備からモデルのデプロイまで、お好みのノートブック環境で数秒で行うことができます。
<br><br>
参照：https://pycaret.org/

<b>特徴①：Increased Productivity</b><br><br>

「PyCaret being a low-code library makes you more productive. With less time spent coding, you and your team can now focus on business problems.」
<br><br>
DeepL翻訳：PyCaret はローコードライブラリであるため、より生産性が高くなります。コーディングに費やす時間が減り、あなたとあなたのチームはビジネス上の問題に集中できるようになりました。

<b>特徴②：Easy to Use</b><br><br>

「PyCaret is simple and easy to use machine learning library that will help you to perform end-to-end ML experiments with less lines of code.」
<br><br>
DeepL翻訳：PyCaretはシンプルで使いやすい機械学習ライブラリで、少ないコード行数でエンドツーエンドのML実験を行うことができます。

<b>特徴③：Business Ready</b><br><br>

「PyCaret is a business ready solution. It allows you to do prototyping quickly and efficiently from your choice of notebook environment.」
<br><br>
DeepL翻訳：PyCaretはビジネスに即したソリューションです。選んだノートブック環境から素早く効率的にプロトタイピングを行うことができます。

# 出来ること

* クラス分類
* 回帰分析
* クラスタリング
* 異常検知(教師無し学習)
* 自然言語処理
* 相関ルールマイニング

実装例：https://github.com/Kazuhito00/PyCaret-Learn

# とりあえずTitanic

# データ読み込み

In [ ]:
import numpy as np 
import pandas as pd 
import pandas_profiling

In [ ]:
train_data = pd.read_csv('./input/titanic/train.csv')
test_data  = pd.read_csv('./input/titanic/test.csv')
submission_data = pd.read_csv('./input/titanic/gender_submission.csv')

In [ ]:
train_data.info()

In [ ]:
train_data.profile_report()

# 今回はEDA、特徴量エンジニアリング省略。。。

# データセットアップ

In [ ]:
# クラス分類用インポート
from pycaret.classification import *

In [ ]:
exp = setup(data=train_data,   # 訓練用データ
            target='Survived', # 目標変数
            ignore_features = ['PassengerId', 'Name'],   # 訓練に使用しない説明変数
            session_id=42,     # 乱数シード
            silent=True)

# モデル比較

In [ ]:
compare_models(fold=5)

# モデル生成

In [ ]:
lgbm_model = create_model('lightgbm')

# モデルチューニング

In [ ]:
tune_lgbm_model = tune_model('lr', optimize='Recall')

# プロット

In [ ]:
evaluate_model(lgbm_model)

In [ ]:
interpret_model(lgbm_model)

# アンサンブル学習（ブレンディング）

In [ ]:
blend_model = blend_models(fold=5)  # モデル未指定時はサポートするモデル全てでブレンディング

# アンサンブル学習（スタッキング）

In [ ]:
rf_model = create_model('rf', verbose=False)
dt_model = create_model('dt', verbose=False)
stack_model = stack_models([rf_model, dt_model, lgbm_model], fold=5)

# アンサンブル学習（バギング）

In [ ]:
ensembled_model = ensemble_model(dt_model)

# アンサンブル学習（ブースティング）

In [ ]:
ensembled_model = ensemble_model(dt_model, method='Boosting')

# 推論

In [ ]:
predictions = predict_model(lgbm_model, data=test_data)
predictions.head()

# チャレンジ中のコンペ

* University of Liverpool - Ion Switching<br>
細胞の電気シグナルを元にイオンチャンネルの開放数を予想するコンペ

* M5 Forecasting - Accuracy<br>
ウォルマートの商品毎の売り上げ個数を予想するコンペ

一晩、学習寝かせて、さっき見たら<br>スコア激下がりしてた、、、ミスったorz

# 所感

* (一部の)前処理、学習、推論が簡潔なコードで書ける<br>
とりあえずLightGBM学習回して<br>interpret_model()で重要特徴量見るのとか瞬殺
* (当然ですが)EDAや特徴量エンジニアリングが大事<br>
でも、ある程度までは自動でそこそこ精度が上がる
<br><br>
* 初心者向けでも上級者向けでも無く、<br>割り切った中級者向けな印象(後輩くんにはどうだろう🤔)

<center><font size="12" color="#efeff0"><b>ご清聴ありがとうございました🙃</b></font></center>